In [ ]:
# __QUADROTORSIMPARAMS__
# This file implement the sim parameters
# simple quadrotor simulation tool

In [ ]:
import import_ipynb
from quadrotor_dynamics import QuadrotorDynamics
import numpy as np
import random
import time
import matplotlib.pyplot as plt

In [ ]:
TURNS = 3

In [ ]:
class SimulationParams:
    def __init__(self):
        self.mass = 1
        self.Ixx = 0.0053
        self.length = 0.2
        self.Bup = 21.58
        self.Bdown = 3.92
        self.Cpmax = np.pi * 1800/180
        self.Cn = TURNS
        self.gravity = 9.81
    
    def get_acceleration(self,p0, p3):
        ap = {
            'acc': (-self.mass * self.length * (self.Bup - p0) / (4 * self.Ixx)),
            'start': (self.mass * self.length * (self.Bup - self.Bdown) / (4 * self.Ixx)),
            'coast': 0,
            'stop': (-self.mass * self.length * (self.Bup - self.Bdown) / (4 * self.Ixx)),
            'recover': (self.mass * self.length * (self.Bup - p3) / (4 * self.Ixx))
        }
        return ap
    
    def get_inintial_parameters(self):
        p0 = p3 = 0.9 * self.Bup
        p1 = p4 = 0.1
        acc_start = self.get_acceleration(p0, p3)['start']
        p2 = (2 * np.pi * self.Cn / self.Cpmax) - (self.Cpmax / acc_start)
        return [p0, p1, p2, p3, p4]
        
    def get_sections(self, parameters):
        sections = np.zeros(5, dtype='object')
        [p0, p1, p2, p3, p4] = parameters
        
        ap = self.get_acceleration(p0, p3)
        
        T2 = (self.Cpmax - p1 * ap['acc']) / ap['start']
        T4 = -(self.Cpmax + p4 * ap['recover']) / ap['stop']
        
        aq = 0
        ar = 0
        
        sections[0] = (self.mass * p0, [ap['acc'], aq, ar], p1)
        
        temp = self.mass * self.Bup - 2 * abs(ap['start']) * self.Ixx / self.length
        sections[1] = (temp, [ap['start'], aq, ar], T2)
        
        sections[2] = (self.mass * self.Bdown, [ap['coast'], aq, ar], p2)
        
        temp = self.mass * self.Bdown - 2 * abs(ap['stop'] * self.Ixx / self.length)
        sections[3] = (temp, [ap['stop'], aq, ar], T4)
        
        sections[4] = (self.mass * p3, [ap['recover'], aq, ar], p4)
        
        return sections
    

In [ ]:
def fly_quadrotor(params=None):
    gen = SimulationParams()
    quadrotor = QuadrotorDynamics()
    if not params:
        params = gen.get_inintial_parameters()
    sections = gen.get_sections(params)
    state = quadrotor.update_state(sections)
    draw_plots(state)

In [ ]:
def draw_plots(states):
    # rename for readability
    x = states[:, 0]    # x -position
    y = states[:, 1]    # y -position
    z = states[:, 2]    # z -position
    u = states[:, 3]    # velocity in x direction
    v = states[:, 4]    # velocity in y direction
    w = states[:, 5]    # velocity in z direction
    phi = states[:, 6]    # phi, roll angle
    theta = states[:, 7]    # theta, pitch angle
    psi = states[:, 8]    # psi, yaw angle
    p = states[:, 9]    # roll rate
    q = states[:, 10]    # pitch rate
    r = states[:, 11]    # yaw rate
    
    plt.figure(figsize=(16,9))
    plt.subplot(421)
    plt.plot(x)
    plt.xlabel('time')
    plt.ylabel('x-position')
    plt.grid()
    plt.subplot(422)
    plt.plot(y)
    plt.xlabel('time')
    plt.ylabel('y-position')
    plt.grid()
    plt.subplot(423)
    plt.plot(z)
    plt.xlabel('time')
    plt.ylabel('z-position')
    plt.grid()
    plt.subplot(424)
    plt.plot(u)
    plt.xlabel('time')
    plt.ylabel('u-velocity')
    plt.grid()
    plt.subplot(425)
    plt.plot(v)
    plt.xlabel('time')
    plt.ylabel('v-velocity')
    plt.grid()
    plt.subplot(426)
    plt.plot(w)
    plt.xlabel('time')
    plt.ylabel('w-velocity')
    plt.grid()
    plt.figure(figsize=(16,9))
    plt.subplot(421)
    plt.plot(phi)
    plt.xlabel('time')
    plt.ylabel('phi')
    plt.grid()
    plt.subplot(422)
    plt.plot(theta)
    plt.xlabel('time')
    plt.ylabel('theta')
    plt.grid()
    plt.subplot(423)
    plt.plot(psi)
    plt.xlabel('time')
    plt.ylabel('psi')
    plt.grid()
    plt.subplot(424)
    plt.plot(p)
    plt.xlabel('time')
    plt.ylabel('p - roll rate')
    plt.grid()
    plt.subplot(425)
    plt.plot(q)
    plt.xlabel('time')
    plt.ylabel('q - pitch rate')
    plt.grid()
    plt.subplot(426)
    plt.plot(r)
    plt.xlabel('time')
    plt.ylabel('r - yaw rate')
    plt.grid()
    
    plt.show()